# `tensorflow.keras` transfer learning from pretrained weights


This notebook demonstrates the use of (1) a well-known CNN architecture and (2) pretrained weights on `RadImageNet` in a multiclassification task.

References:

* The pretrained weights published by [Mei et al (2022)](https://pubs.rsna.org/doi/10.1148/ryai.210315) in their github [link](https://github.com/BMEII-AI/RadImageNet).

* The github repository form the BAGLS team contained in this github [link](https://github.com/anki-xyz/bagls/blob/master/Utils/DataGenerator.py#L109)


In [1]:
# dev convenience
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("..")
import PATHS

import os
import numpy as np

# os.environ["WANDB_SILENT"] = "True"
os.environ["WANDB_NOTEBOOK_NAME"] = "04-tf-pretrained-cat.ipynb"

PROJECT_NAME = 'bagls-sh-project'
RUN_NAME = 'ResNet50-notebook-test'
METRICS_TABLE_NAME = 'metrics_table'
GRADCAM_LAYER_NAME = "conv5_block3_out"

In [3]:
import wandb
print("W&B: ", wandb.__version__)
wandb.login()

# # manage logs
# import logging

# logger = logging.getLogger("wandb")
# logger.setLevel(logging.ERROR)

# logging.getLogger('tensorflow').disabled = True

W&B:  0.13.5


wandb: Currently logged in as: miked. Use `wandb login --relogin` to force relogin


True

In [4]:
# tf loader
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow import keras
tf.keras.backend.clear_session()
os.environ["CUDA_VISIBLE_DEVICES"] = "2,3"

In [5]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5196010706496549164
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10925703168
locality {
  bus_id: 1
  links {
    link {
      device_id: 1
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 13368640541295084915
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:04:00.0, compute capability: 6.1"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 10925703168
locality {
  bus_id: 1
  links {
    link {
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 11555287387960119255
physical_device_desc: "device: 1, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:08:00.0, compute capability: 6.1"
, name: "/device:GPU:2"
device_type: "GPU"
memory_limit: 10925703168
locality {
  bus_id: 2
  numa_node: 1
  links {
    link {
      device_id: 3
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 

2022-12-01 03:49:50.359245: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-01 03:49:53.154913: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /device:GPU:0 with 10419 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:04:00.0, compute capability: 6.1
2022-12-01 03:49:53.155758: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /device:GPU:1 with 10419 MB memory:  -> device: 1, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:08:00.0, compute capability: 6.1
2022-12-01 03:49:53.156403: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /device:GPU:2 with 10419 MB memory:  -> device: 2, name: NVIDIA GeForce GTX 1080 Ti, pci

In [6]:
import config
configs = config.nb_configs

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.imagenet_utils import preprocess_input

# initialize data generator
train_datagen = ImageDataGenerator(
    preprocessing_function=None,
    validation_split=configs["validation_split"],
    rescale=configs["rescale"],
    width_shift_range=configs["width_shift_range"],
    height_shift_range=configs["height_shift_range"],
    shear_range=configs["shear_range"],
    zoom_range=configs["zoom_range"],
    fill_mode=configs["fill_mode"],
    horizontal_flip=configs["horizontal_flip"],
    rotation_range=configs["rotation_range"],
)

test_datagen = ImageDataGenerator(
    preprocessing_function=None, 
    rescale=configs["rescale"],
)

In [8]:
train_dir = configs["train_dir"]
test_dir = configs["test_dir"]

batch_size = configs["batch_size"]
class_names = configs["class_names"]
interpol = configs["interpol"]
cmap = configs["cmap"]
label_mode = configs["label_mode"]
labels = configs["labels"]
image_size = configs["image_size"]


train_dataset = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=image_size,
    color_mode=cmap,
    classes=class_names,
    class_mode=label_mode,
    batch_size=batch_size,
    interpolation=interpol,
    subset="training",
)

val_dataset = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=image_size,
    color_mode=cmap,
    classes=class_names,
    class_mode=label_mode,
    batch_size=batch_size,
    interpolation=interpol,
    subset="validation",
)

test_dataset = test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=image_size,
    color_mode=cmap,
    classes=class_names,
    class_mode=label_mode,
    batch_size=batch_size,
    interpolation=interpol,
    shuffle=False, # do not shuffle for later evaluation, alphanum sort
)

configs.update({"val_steps": val_dataset.samples // configs["batch_size"]})

Found 52393 images belonging to 2 classes.
Found 2757 images belonging to 2 classes.
Found 3300 images belonging to 2 classes.


In [9]:
dropout_rate = 0.2
def define_model(pretrained):
    conv_base = pretrained(
        include_top=False,
        weights=PATHS.resnet50_weights,
        input_shape=(*image_size, 3),
        pooling='avg',
    )
    print("Num trainable at load:", len(conv_base.trainable_weights)) 
    conv_base.trainable = False
    print("Num trainable:", len(conv_base.trainable_weights)) 
    
    x = conv_base.output
    
    # layers at this stage are arbitrary
    # can be subjected to hyperparam tuning
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(units=512, activation='relu')(x)
    x = keras.layers.Dropout(configs["dropout_rate"])(x)
    outputs = keras.layers.Dense(units=2, activation="softmax")(x)
    model = keras.Model(inputs=conv_base.input, outputs=outputs)
    return model

In [10]:
pretrained = keras.applications.ResNet50
model = define_model(pretrained)

2022-12-01 00:40:23.096152: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10419 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:04:00.0, compute capability: 6.1
2022-12-01 00:40:23.096533: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 10419 MB memory:  -> device: 1, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:08:00.0, compute capability: 6.1
2022-12-01 00:40:23.096880: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 10419 MB memory:  -> device: 2, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:84:00.0, compute capability: 6.1
2022-12-01 00:40:23.097225: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 10419 MB memory:  -> device: 3, name: NVIDIA GeForce

Num trainable at load: 212
Num trainable: 0


In [41]:
from tensorflow.keras import metrics
import custom_metrics
thresh = configs["thresh"]
metrics_dict = {
    "ACC":  metrics.CategoricalAccuracy(name="ACC"),
    "AUC-ROC": custom_metrics.MulticlassAUC(name='ROC', curve='ROC', pos_label=1),
    "AUC-PR": custom_metrics.MulticlassAUC(name='PR', curve='PR', pos_label=1),
    "TP": custom_metrics.MulticlassTruePositives(name="TP", pos_label=1, thresholds=thresh),
    "TN": custom_metrics.MulticlassTrueNegatives(name="TN", pos_label=1, thresholds=thresh),
    "FP": custom_metrics.MulticlassFalsePositives(name="FP", pos_label=1, thresholds=thresh),
    "FN": custom_metrics.MulticlassFalseNegatives(name="FN", pos_label=1, thresholds=thresh),
}

In [12]:
from tensorflow.keras import optimizers
from tensorflow.keras import losses

# opt = optimizers.Adam(learning_rate=1e-06)
opt = optimizers.Adam()
met = list(metrics_dict.values())

model.compile(
    loss=losses.CategoricalCrossentropy(),
    optimizer=opt,
    metrics=met,
)

In [13]:
# verify arch
# base predictions with untrained classif head
base_preds = model.predict(test_dataset)
base_preds

2022-12-01 00:40:33.458339: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-12-01 00:40:35.964450: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8400


array([[0.5176156 , 0.48238438],
       [0.47994995, 0.52005005],
       [0.53178227, 0.4682177 ],
       ...,
       [0.5324806 , 0.46751934],
       [0.5642422 , 0.43575776],
       [0.503941  , 0.49605897]], dtype=float32)

In [14]:
from interpretation import ValLog, GRADCamLogger

# initialize run
run = wandb.init(
    project=PROJECT_NAME, 
    name=RUN_NAME,
    config=configs, 
    job_type='train',
)

wandb_callback = wandb.keras.WandbCallback(
    monitor="val_ROC",
    mode="max",
    save_model=True,
    save_graph=True,
    compute_flops=True,
)

callbacks = [
    wandb_callback,
    ValLog(generator=val_dataset, num_log_batches=1),
#     GRADCamLogger(generator=test_dataset, 
#                   layer_name=GRADCAM_LAYER_NAME, num_log_batches=1)
]

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


In [15]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=configs["epochs"], 
    shuffle=True,
    callbacks=callbacks,
)
run.finish()

2022-12-01 00:41:12.091625: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 4
2022-12-01 00:41:12.091879: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-12-01 00:41:12.101369: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10419 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:04:00.0, compute capability: 6.1
2022-12-01 00:41:12.101919: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 10419 MB memory:  -> device: 1, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:08:00.0, compute capability: 6.1
2022-12-01 00:41:12.102486: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 10419 MB memory:  -> device: 2, name: NVIDIA GeForce GTX 1080 Ti, pc

Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
Epoch 1/10
410/410 [==============================] - 872s 2s/step - loss: 0.5088 - ACC: 0.7427 - ROC: 0.8018 - PR: 0.7495 - TP: 10287.0000 - TN: 28625.0000 - FP: 3628.0000 - FN: 9853.0000 - val_loss: 0.4309 - val_ACC: 0.8012 - val_ROC: 0.8802 - val_PR: 0.8383 - val_TP: 744.0000 - val_TN: 1465.0000 - val_FP: 232.0000 - val_FN: 316.0000


/home/mdorosan/.conda-envs/msds2022-ml3/lib/python3.9/site-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
2022-12-01 00:55:58.868219: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /home/mdorosan/2022/bagls-sh-project/notebooks/wandb/run-20221201_004057-3j2se4wx/files/model-best/assets


/home/mdorosan/.conda-envs/msds2022-ml3/lib/python3.9/site-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
wandb: Adding directory to artifact (/home/mdorosan/2022/bagls-sh-project/notebooks/wandb/run-20221201_004057-3j2se4wx/files/model-best)... Done. 0.7s


...

Epoch 9/10
410/410 [==============================] - 853s 2s/step - loss: 0.1206 - ACC: 0.9546 - ROC: 0.9914 - PR: 0.9873 - TP: 18766.0000 - TN: 31246.0000 - FP: 1007.0000 - FN: 1374.0000 - val_loss: 0.0944 - val_ACC: 0.9666 - val_ROC: 0.9960 - val_PR: 0.9938 - val_TP: 987.0000 - val_TN: 1678.0000 - val_FP: 19.0000 - val_FN: 73.0000
INFO:tensorflow:Assets written to: /home/mdorosan/2022/bagls-sh-project/notebooks/wandb/run-20221201_004057-3j2se4wx/files/model-best/assets


/home/mdorosan/.conda-envs/msds2022-ml3/lib/python3.9/site-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
wandb: Adding directory to artifact (/home/mdorosan/2022/bagls-sh-project/notebooks/wandb/run-20221201_004057-3j2se4wx/files/model-best)... Done. 0.7s


Epoch 10/10
410/410 [==============================] - 859s 2s/step - loss: 0.1076 - ACC: 0.9604 - ROC: 0.9932 - PR: 0.9895 - TP: 18957.0000 - TN: 31360.0000 - FP: 893.0000 - FN: 1183.0000 - val_loss: 0.0839 - val_ACC: 0.9735 - val_ROC: 0.9968 - val_PR: 0.9953 - val_TP: 1005.0000 - val_TN: 1679.0000 - val_FP: 18.0000 - val_FN: 55.0000
INFO:tensorflow:Assets written to: /home/mdorosan/2022/bagls-sh-project/notebooks/wandb/run-20221201_004057-3j2se4wx/files/model-best/assets


/home/mdorosan/.conda-envs/msds2022-ml3/lib/python3.9/site-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
wandb: Adding directory to artifact (/home/mdorosan/2022/bagls-sh-project/notebooks/wandb/run-20221201_004057-3j2se4wx/files/model-best)... Done. 1.1s


ACC,▁▄▅▆▇▇▇███
FN,█▅▄▃▂▂▁▁▁▁
FP,█▇▅▄▃▂▂▂▁▁
PR,▁▄▆▇▇█████
ROC,▁▅▆▇▇█████
TN,▁▂▄▅▆▇▇▇██
TP,▁▄▅▆▇▇████
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▆▅▃▃▂▂▂▁▁
val_ACC,▁▄▅▆▆▇▇███
val_FN,█▃▁▅▅▁▂▁▁▁


## End